In [1]:
# import the library
import sumolib
import random
import traci  # noqa
import sys
import os

In [3]:
DUAROUTER = sumolib.checkBinary('duarouter')

In [4]:
class GeoCoordinates:
    """
    """

    def __init__(self, X=None, Y=None):
        self.X = X
        self.Y = Y

    def getCoordinates(self):
        return {"X": self.X, "Y": self.Y}

    def setCoordinates(self, X, Y):
        self.X = X
        self.Y = Y

    def getCoordinatesOSM(self):
        return self.Y, self.X

In [5]:
class Route:
    def __init__(self, from_coord, to_coord, sumo_net):
        self.fromCoord = from_coord
        self.toCoord = to_coord
        self.sumo_net = sumo_net


    def getClosestEdge(self, coordinates):
        x, y = self.sumo_net.convertLonLat2XY(coordinates.Y, coordinates.X)
        edges = net.getNeighboringEdges(x, y, 10)
        closest_edge = None
        if len(edges) > 0:
            distances_and_edges = sorted([(dist, edge) for edge, dist in edges])
            dist, closest_edge = distances_and_edges[0]
        return closest_edge


    def getShortestPath(self):

        fromEdge = self.getClosestEdge(self.fromCoord)
        toEdge = self.getClosestEdge(self.toCoord)

        if fromEdge and toEdge:
            return self.sumo_net.getShortestPath(fromEdge, toEdge)
        return None


In [6]:
class Vehicle:
    # id="typeEVP" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="3" maxSpeed="30" guiShape="police"
    def __init__(self, guiShape='passenger', id='default', accel=0.8, decel=4.5, sigma=0.5, length=5, minGap=3, maxSpeed=16.67):
        self.guiShape = guiShape
        self.id = id
        self.accel = accel
        self.decel = decel
        self.sigma = sigma
        self.length = length
        self.minGap = minGap
        self.maxSpeed = maxSpeed

    def getSumoTag(self):
        return f'   <vType id="{self.id}" guiShape="{self.guiShape}" accel="{self.accel}" decel="{self.decel}" sigma="{self.sigma}" length="{self.length}" minGap="{self.minGap}" maxSpeed="{self.maxSpeed}" />\n'

In [7]:
net = sumolib.net.readNet('../phd_ev/examples/phd_ev/fsa.net.xml/fsa.net.xml')

In [8]:
coordinates = [
    GeoCoordinates(-12.2630,-38.9475), # Início da PD
    GeoCoordinates(-12.2596,-38.9659), # Final da PD
    GeoCoordinates(-12.2468,-38.9544), # Maria quitéria NS Início
    GeoCoordinates(-12.2684,-38.9553), # Maria quitéria NS Fim
    GeoCoordinates(-12.2630,-38.9549), # Maria quitéria SN Início
    GeoCoordinates(-12.2489,-38.9543), # Maria quitéria SN Fim
    GeoCoordinates(-12.2536,-38.9610),
    GeoCoordinates(-12.2609,-38.9494),
    GeoCoordinates(-12.2629,-38.9589),
    GeoCoordinates(-12.2538,-38.9472),
]

In [9]:
route = Route(coordinates[4], coordinates[0], net)

route.getShortestPath()[0]

(<edge id="34658849#0" from="403261873" to="4954002299"/>,
 <edge id="34658849#1" from="4954002299" to="404421631"/>,
 <edge id="34658849#2" from="404421631" to="403261875"/>,
 <edge id="34658849#3" from="403261875" to="6680732499"/>,
 <edge id="34658849#4" from="6680732499" to="6680732492"/>,
 <edge id="34717023#0" from="6680732492" to="404421710"/>,
 <edge id="34717023#1" from="404421710" to="cluster_403261694_405030298"/>,
 <edge id="172385258#0" from="cluster_403261694_405030298" to="405030335"/>,
 <edge id="172385258#1" from="405030335" to="405030341"/>,
 <edge id="172385258#2" from="405030341" to="405030348"/>,
 <edge id="172385258#3" from="405030348" to="cluster_1832928650_405030295"/>,
 <edge id="155032501#0" from="cluster_1832928650_405030295" to="405030353"/>,
 <edge id="155032501#1" from="405030353" to="405030356"/>,
 <edge id="155032501#2" from="405030356" to="405030392"/>,
 <edge id="155032501#3" from="405030392" to="405030293"/>,
 <edge id="668381497" from="405030293" to=

In [10]:
vehicles = [
    Vehicle(guiShape='emergency', id='v_emergency', maxSpeed=30),
    Vehicle(guiShape='firebrigade', id='v_firebrigade', maxSpeed=28, length=7),
    Vehicle(guiShape='police', id='v_police', maxSpeed=28),
]

In [11]:
n_routes = 4
n_vehicles = 3600

random.seed(4)

with open('data/fsa.rou.xml', 'w') as outf:
        sumolib.xml.writeHeader(outf, root="routes")
        v_id = 0
        outf.write(Vehicle(guiShape='passenger', id='v_passenger').getSumoTag())
        outf.write(Vehicle(guiShape='bus', id='v_bus', length=10).getSumoTag())
        for v in vehicles:
            outf.write(v.getSumoTag())

        routes = []
        main_route = Route(coordinates[0], coordinates[1], net).getShortestPath()
        outf.write(f'   <route cost="{main_route[1]}" id="0" edges="{" ".join([e.getID() for e in main_route[0]])}" />\n')
        for id in range(1, n_routes):
            route = Route(random.choice(coordinates), random.choice(coordinates), net).getShortestPath()
            if route:
                routes.append(route)
                outf.write(f'   <route cost="{route[1]}" id="{id}" edges="{" ".join([e.getID() for e in route[0]])}" />\n')

        vId = 0
        for i in range(n_vehicles):
            route =  random.randint(0, len(routes)-1)
            type = 'passenger'

            p_passenger = 1. / 5
            p_bus = 1. / 90
            p_emergency = 1. / 200

            if random.uniform(0, 1) < p_passenger:
                outf.write(f'   <vehicle id="{vId}" depart="{float(i)}" route="{route}" type="v_passenger"/>\n')
                vId+=1
            if random.uniform(0, 1) < p_emergency:
                vehicle = random.choice(vehicles)
                outf.write(f'   <vehicle id="{vId}" depart="{float(i)}" route="0" type="{vehicle.id}"/>\n')
                vId+=1
            if random.uniform(0, 1) < p_bus:
                outf.write(f'   <vehicle id="{vId}" depart="{float(i)}" route="{route}" type="v_bus" color="0,1,0"/>\n')
                vId+=1

        outf.write('</routes>\n')


In [12]:
sumo_home = os.environ['SUMO_HOME'] = os.path.join('/opt', 'sumo')
sys.path.append(sumo_home)
sys.path.append(os.path.join(sumo_home, 'bin'))
sumoBinary = sumolib.checkBinary('sumo-gui')

In [ ]:
def start_traci():
    traci.start([sumoBinary, "-c", "data/fsa.sumocfg",
                                 "--tripinfo-output", "tripinfo.xml"])


In [ ]:
def run():
    """execute the TraCI control loop"""
    step = 0
    # we start with phase 2 where EW has green
    tl_id = "cluster_1674752281_1674752283_403261502_403261510_403261862_403261865"
    traci.trafficlight.setPhase(tl_id, 2)
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        if traci.trafficlight.getPhase(tl_id) == 2:
            # we are not already switching
            if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
                # there is a vehicle from the north, switch
                traci.trafficlight.setPhase(tl_id, 3)
            else:
                # otherwise try to keep green for EW
                traci.trafficlight.setPhase(tl_id, 2)
        step += 1
    traci.close()
    sys.stdout.flush()


In [ ]:
start_traci()

In [ ]:
run()

In [ ]:
traci.close()


In [24]:
ev_from = random.choice(net.getEdges())
ev_to = random.choice(net.getEdges())

path = net.getShortestPath(ev_from, ev_to)
path

((<edge id="828140822" from="6680844022" to="3747825735"/>,
  <edge id="828140821" from="3747825735" to="5258659316"/>,
  <edge id="828140820" from="5258659316" to="4953774754"/>,
  <edge id="166825502" from="4953774754" to="7227957994"/>,
  <edge id="849619222#0" from="7227957994" to="1782569259"/>,
  <edge id="849619222#1" from="1782569259" to="3747825740"/>,
  <edge id="849619222#2" from="3747825740" to="1782569263"/>,
  <edge id="849619222#3" from="1782569263" to="3747825738"/>,
  <edge id="849619222#4" from="3747825738" to="403261835"/>,
  <edge id="849619222#5" from="403261835" to="403261839"/>,
  <edge id="849619222#6" from="403261839" to="407583449"/>,
  <edge id="849619222#7" from="407583449" to="405030246"/>,
  <edge id="849619223#0" from="405030246" to="407583484"/>,
  <edge id="34812970#0" from="407583484" to="407583475"/>,
  <edge id="371191305#3" from="407583475" to="407583478"/>,
  <edge id="267485837#0" from="407583478" to="4045165711"/>,
  <edge id="267485837#1" from="

In [25]:
from enum import Enum
class EventVehicleType(Enum):
    EMERGENCY = Vehicle(guiShape='emergency', id='v_emergency', maxSpeed=30)
    FIRE = Vehicle(guiShape='firebrigade', id='v_firebrigade', maxSpeed=28, length=7)
    POLICE = Vehicle(guiShape='police', id='v_police', maxSpeed=28)

'v_emergency'